**Assignment 5**

1.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import preprocessing

loc='/content/drive/MyDrive/Other-Material/'

boston = datasets.load_boston(return_X_y=False)
df = pd.DataFrame(boston.data, columns=boston.feature_names)
df['target'] = boston.target
mat=df.to_numpy()
print(boston.DESCR)
print("Shape of matrix loaded:",mat.shape)
print("Number of Features:",mat.shape[1]-1)
print("Number of Patterns:",mat.shape[0])


.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

a)

In [2]:
X=mat[:,:-1]
np.random.shuffle(X)  
X = preprocessing.normalize(X)

#X = X / X.max(axis=0)
X=np.column_stack((np.ones(X.shape[0]),X))
Y=mat[:,-1]
Y=Y.reshape(mat.shape[0],1)
train_size=int(X.shape[0]*0.3)
val_size=int(X.shape[0]*0.1)
X_train,Y_train=X[0:train_size,:],Y[0:train_size,:]
X_val,Y_val=X[train_size:train_size+val_size,:],Y[train_size:train_size+val_size,:]
X_test,Y_test=X[train_size+val_size:,:],Y[train_size+val_size:,:]
print(X.shape,Y.shape)
print(X_train.shape,Y_train.shape)
print(X_val.shape,Y_val.shape)
print(X_test.shape,Y_test.shape)

(506, 14) (506, 1)
(151, 14) (151, 1)
(50, 14) (50, 1)
(305, 14) (305, 1)


In [3]:
def  cal_cost(w,X,y):
    m = len(y)
    predictions = X.dot(w)
    cost = (1/2) * np.sum(np.square(predictions-y))/m
    return cost


In [4]:
def gradient_descent(X,y,theta,learning_rate=0.001,iterations=100,p=0.3):
    m = len(y)
    j_hist=[]
    prev=-1
    cost=0
    for it in range(iterations):
        cost=0
        prediction = np.dot(X,theta)
        theta = theta -(1/m)*learning_rate*( X.T.dot((prediction - y)))
        cost = cal_cost(theta,X,y)
        j_hist.append(cost)
        if abs(cost-prev)<p:
          break
        prev=cost
        
    return j_hist,theta

In [5]:
def stocashtic_gradient_descent(X,y,w,learning_rate=0.001,iterations=100,rho=0.3):
    m = len(y)
    prev=-1
    j_hist=[]
    cost=0
    for it in range(iterations):
        cost =0.0
        for i in range(m):
            X_i = X[i,:].reshape(1,X.shape[1])
            y_i = y[i].reshape(1,1)
            prediction = np.dot(X_i,w)
            w = w-learning_rate*( X_i.T.dot((prediction - y_i)))
            cost += cal_cost(w,X_i,y_i)
        if abs(prev-cost)<rho:
          break
        j_hist.append(cost)
        prev=cost
    return j_hist,w

In [6]:
alpha=[0.1,0.01,0.02,0.03,0.001]
rho=[0.5,0.6,0.7,0.8,0.9]
epoch=[10,20,30,40,50]


In [7]:
w_ori= np.random.randn(X.shape[1],1)
MSE=np.zeros(shape=(len(alpha)))
for i in range(len(alpha)):
  jH,w= stocashtic_gradient_descent(X_val,Y_val,w_ori,alpha[i],epoch[i],rho[i])
  MSE[i]=jH[-1]
print(MSE)
MSE_min=min(MSE)
best, = np.where(np.isclose(MSE, MSE_min))
best_param = best[0]
print('Best values are at:',best_param)

[1588.74048724 2375.75063975 2280.09001065 2200.8370448  2434.66325083]
Best values are at: 0


**I**

In [8]:
df = pd.DataFrame (MSE)
filepath = '/content/drive/MyDrive/Other-Material/1stpt.xlsx'
df.to_excel(filepath, index=False)

FileNotFoundError: ignored

**II**

In [ ]:
all_MSE=[0,0,0]
w_ori= np.random.randn(X.shape[1],1)
j_train,w= stocashtic_gradient_descent(X_train,Y_train,w_ori,alpha[best_param],epoch[best_param],rho[best_param])
all_MSE[0]=j_train[-1]
all_MSE[1]=cal_cost(w,X_test,Y_test)
all_MSE[2] = cal_cost(w,X_val,Y_val)
df = pd.DataFrame (all_MSE)
filepath = '/content/drive/MyDrive/Other-Material/2ndpt.xlsx'
df.to_excel(filepath, index=False)

**III**

In [ ]:
!pip install XlsxWriter
import xlsxwriter

In [ ]:
x_epoch=np.arange(0,len(j_train))
fig=plt.figure()
plt.plot(x_epoch,j_train)
plt.xlabel('num_epoch')
plt.ylabel('mse')
plt.title('num_epoch vs mse for train') 
plt.show() 
fig.savefig(loc+'python_plot.png')
writer = pd.ExcelWriter(loc+'3rdpt.xlsx', engine='xlsxwriter')
workbook = writer.book
worksheet = workbook.add_worksheet('Haha')
worksheet.insert_image('A1', loc+'python_plot.png')
workbook.close
writer.save()
'''writer = pd.ExcelWriter(loc+'3rdpt.xlsx', engine='xlsxwriter')

df.to_excel(writer, sheet_name='Sheet1')
workbook  = writer.book
worksheet = writer.sheets['Sheet1']

# Insert an image.
worksheet.insert_image('A1', loc+'python_plot.png')

# Close the Pandas Excel writer and output the Excel file.
writer.save()'''